In [1]:
import math
import re

from tqdm import tqdm

In [2]:
with open("example.txt") as f:
    lines_example = f.read().splitlines()

In [3]:
with open("input.txt") as f:
    lines = f.read().splitlines()

In [4]:
def get_seeds_maps(lines: list[str]) -> list[dict]:
    seeds = []
    maps = []

    i = 0
    while i < len(lines):
        line = lines[i]
        print(line)

        if line == "":
            print("skipped")
            i += 1
            continue

        if "seeds" in line:
            seeds = [int(e) for e in re.split(r" +", line.split(":")[1].strip())]

        if "map" in line:
            print(line)
            map_dict = {}
            i += 1

            while i < len(lines) and lines[i] != "":
                line = lines[i]
                print("processing : ", line)
                line_list = re.split(r" +", line.strip())

                start_destination = int(line_list[0])
                start_source = int(line_list[1])
                length = int(line_list[2])

                map_dict[range(start_source, start_source + length)] = range(
                    start_destination, start_destination + length
                )
                i += 1
            maps.append(map_dict)

        i += 1
    return seeds, maps

In [5]:
seeds_example, maps_example = get_seeds_maps(lines_example)
seeds, maps = get_seeds_maps(lines)

seeds: 79 14 55 13

skipped
seed-to-soil map:
seed-to-soil map:
processing :  50 98 2
processing :  52 50 48
soil-to-fertilizer map:
soil-to-fertilizer map:
processing :  0 15 37
processing :  37 52 2
processing :  39 0 15
fertilizer-to-water map:
fertilizer-to-water map:
processing :  49 53 8
processing :  0 11 42
processing :  42 0 7
processing :  57 7 4
water-to-light map:
water-to-light map:
processing :  88 18 7
processing :  18 25 70
light-to-temperature map:
light-to-temperature map:
processing :  45 77 23
processing :  81 45 19
processing :  68 64 13
temperature-to-humidity map:
temperature-to-humidity map:
processing :  0 69 1
processing :  1 0 69
humidity-to-location map:
humidity-to-location map:
processing :  60 56 37
processing :  56 93 4
seeds: 2494933545 159314859 4045092792 172620202 928898138 554061882 2740120981 81327018 2031777983 63513119 2871914181 270575980 2200250633 216481794 3289604059 25147787 3472625834 10030240 260990830 232636388

skipped
seed-to-soil map:


# Part 1

In [59]:
def search_in_keys(to_search: int, keys: range) -> int:
    # print(to_search,keys)
    for key in keys:
        if search in key:
            range_to_search = map_[key]

            return range_to_search.start + (search - key.start)
    return to_search

In [60]:
map_index = 0

min_values = []
for seed in seeds:
    search = seed
    for i, map_ in enumerate(maps):
        if i == (len(maps) - 1):
            min_values.append(search_in_keys(search, map_.keys()))
        else:
            search = search_in_keys(search, map_.keys())

In [62]:
min(min_values)

165788812

# Part 2

In [6]:
ranges_to_search = [range(a, a + b) for a, b in zip(seeds[:-1:2], seeds[1::2])]
ranges_to_search_example = [
    range(a, a + b) for a, b in zip(seeds_example[:-1:2], seeds_example[1::2])
]

In [7]:
maps_example

[{range(98, 100): range(50, 52), range(50, 98): range(52, 100)},
 {range(15, 52): range(0, 37),
  range(52, 54): range(37, 39),
  range(0, 15): range(39, 54)},
 {range(53, 61): range(49, 57),
  range(11, 53): range(0, 42),
  range(0, 7): range(42, 49),
  range(7, 11): range(57, 61)},
 {range(18, 25): range(88, 95), range(25, 95): range(18, 88)},
 {range(77, 100): range(45, 68),
  range(45, 64): range(81, 100),
  range(64, 77): range(68, 81)},
 {range(69, 70): range(0, 1), range(0, 69): range(1, 70)},
 {range(56, 93): range(60, 97), range(93, 97): range(56, 60)}]

In [16]:
def search_in_map(range_to_search: range, map: dict) -> tuple[range, range, range]:
    """Search overlaps between a range and the mapping ranges"""
    overlaps = []
    next_ranges = []
    no_overlaps = []
    for map_range, next_range in map.items():
        overlap = range(
            max(range_to_search[0], map_range[0]),
            min(range_to_search[-1], map_range[-1]) + 1,
        )
        if len(overlap) > 0:
            overlaps.append(overlap)

            left_bound = next_range[0] + abs(overlap[0] - map_range[0])
            right_bound = next_range[-1] - abs(overlap[-1] - map_range[-1])
            next_ranges.append(range(left_bound, right_bound + 1))

            if overlap[0] > range_to_search[0]:
                # Left remaining
                no_overlaps.append(range(range_to_search[0], overlap[0]))
            if range_to_search[-1] > overlap[-1]:
                # Right remaining
                no_overlaps.append(range(overlap[-1] + 1, range_to_search[-1] + 1))


    no_overlaps_clean = [e for e in no_overlaps if e not in overlaps]
    if len(overlaps) > 0:
        return next_ranges, no_overlaps_clean
    else:
        return range_to_search

In [19]:
final_ranges = []


def search_ranges(ranges_to_search: list[range], maps: list[dict], depth: int) -> range:
    """Recursively call search_in_map to find the final ranges after walking the mapping tree"""
    max_depth = len(maps) - 1
    print(max_depth)
    print(depth)
    current_map = maps[depth]
    curr_ranges_to_search = ranges_to_search

    for range_to_search in curr_ranges_to_search:
        res = search_in_map(range_to_search, current_map)

        if isinstance(res, range):
            if depth == max_depth:
                print("end")
                final_ranges.append(res)
            else:
                print("case 1")
                search_ranges([res], maps, depth=depth + 1)
        elif isinstance(res, tuple):
            if depth == max_depth:
                print("end")
                final_ranges.extend(res[0])
                if len(res[1]) > 0:
                    final_ranges.extend(res[1])
            else:
                print("case 2")
                search_ranges(res[0], maps, depth=depth + 1)
                if len(res[1]) > 0:
                    print("case 2 with remaining no overlaps")
                    search_ranges(res[1], maps, depth=depth)

In [20]:
search_ranges(ranges_to_search,maps,0)

6
0
case 2
6
1
case 2
6
2
case 2
6
3
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
4
case 2
6
5
case 2
6
6
end
case 1
6
2
case 2
6
3
case 2
6
4
case 1
6
5
case 2
6
6
end
case 1
6
5
case 2
6
6
end
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2 with remaining no overlaps
6
2
case 2
6
3
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
3
case 2
6
4
case 1
6
5
case 2
6
6
end
case 1
6
5
case 2
6
6
end
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
1
case 1
6
2
case 2
6
3
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2 with remaining no overlaps
6
3
case 1
6
4
case 1
6
5
case 2
6
6
end
case 1
6
2
case 2
6
3
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
5
case 2
6
6
end
case 2
6
5
case 2
6
6
end
case 2 with remaining no overlaps
6
3
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
5
case 2
6
6
end
case 2
6
4
case 2
6
5
case 2
6
6
end
case 2
6
5
case 2
6
6
end
case 2
6
4
case 2
6
5
case 2
6
6
end
ca

In [21]:
final_ranges

[range(2236580069, 2270434865),
 range(2491372752, 2568376954),
 range(838672006, 844911387),
 range(844911387, 865780963),
 range(724752149, 732239063),
 range(1829760280, 1843620270),
 range(724752149, 732239063),
 range(1829760280, 1843620270),
 range(838672006, 844911387),
 range(844911387, 865780963),
 range(724752149, 732239063),
 range(1969590954, 1987065389),
 range(776999018, 782198941),
 range(2270434865, 2330820865),
 range(1935956166, 1948230383),
 range(962007902, 970336878),
 range(2955432722, 2960166673),
 range(962007902, 970336878),
 range(2955432722, 2960166673),
 range(1935956166, 1948230383),
 range(962007902, 970336878),
 range(3916070208, 3919086833),
 range(3916070208, 3919086833),
 range(4095434, 8677039),
 range(1928058, 4095434),
 range(1765894261, 1797120632),
 range(3952597071, 3974085119),
 range(3720509340, 3722067319),
 range(3209925310, 3210110006),
 range(1928058, 4095434),
 range(1765894261, 1797120632),
 range(3952597071, 3974085119),
 range(372050934

In [22]:
min_value = math.inf
for range_ in final_ranges:
    if range_.start<min_value:
        min_value=range_.start
min_value

1928058